In [8]:
import os
import json
from uuid import uuid4
from dotenv import find_dotenv, load_dotenv

# OpenAI client (v1 interface)
from openai import OpenAI

# Firebase Admin SDK
from firebase import initialize_firebase

In [9]:
dotenv_path  = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
# ——— Initialize OpenAI ———
OPENAI_API_KEY = os.getenv("OPENAI_RANDY_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [10]:
db = initialize_firebase()

In [11]:
# ——— Planner Function ———
def plan_research_task(task_description: str) -> dict:
    """
    Calls GPT-4 to break a research goal into a JSON plan.
    Returns a Python dict matching the plan schema.
    """
    # instruct GPT to output machine-parseable JSON
    system_prompt = (
        "You are an expert research assistant. "
        "Output a JSON object with this schema:\n"
        "{\n"
        '  "goal": string,\n'
        '  "subtasks": [\n'
        "    {\n"
        '      "id": string,        # unique\n'
        '      "type": string,      # e.g. "search", "api", "validate"\n'
        '      "description": string,\n'
        '      "params": object     # free-form parameters for executor\n'
        "    }\n"
        "  ]\n"
        "}\n"
        "Do not wrap the JSON in markdown or any extra text."
    )
    user_prompt = f"The research goal is: \"{task_description}\". What are the logical subtasks to execute it step by step?"

    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system",  "content": system_prompt},
            {"role": "user",    "content": user_prompt}
        ],
        temperature=0.2
    )

    # parse the JSON directly
    plan_json = json.loads(resp.choices[0].message.content)
    # ensure there's a unique plan ID
    plan_json["plan_id"] = str(uuid4())
    return plan_json

# ——— Firebase Storage Function ———
def save_plan_to_firebase(plan: dict):
    """
    Persists the plan dict into Firestore under collection 'research_plans'.
    Document ID will be plan['plan_id'].
    """
    plan_id = plan.get("plan_id") or str(uuid4())
    doc_ref = db.collection("research_plans").document(plan_id)
    doc_ref.set(plan)
    print(f"Saved plan {plan_id} to Firestore.")



In [12]:
task = "Find a list of hospitals in Ontario"
plan = plan_research_task(task)
print("Generated Plan:")
print(json.dumps(plan, indent=2))
save_plan_to_firebase(plan)

Generated Plan:
{
  "goal": "Find a list of hospitals in Ontario",
  "subtasks": [
    {
      "id": "1",
      "type": "search",
      "description": "Search for a reliable source that provides a list of hospitals in Ontario",
      "params": {
        "keywords": [
          "list of hospitals",
          "Ontario"
        ]
      }
    },
    {
      "id": "2",
      "type": "validate",
      "description": "Verify the reliability and credibility of the source",
      "params": {
        "source": ""
      }
    },
    {
      "id": "3",
      "type": "api",
      "description": "Extract the list of hospitals from the source",
      "params": {
        "url": "",
        "data_format": "json"
      }
    },
    {
      "id": "4",
      "type": "validate",
      "description": "Check the completeness and accuracy of the extracted list",
      "params": {
        "data": ""
      }
    }
  ],
  "plan_id": "2cf01aec-8099-41f9-aa36-25a0a8035322"
}
Saved plan 2cf01aec-8099-41f9-aa36-25a0